In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd
from gPhoton import galextools as gt

In [ ]:
# Import the function definitions that accompany this notebook tutorial.
nb_funcdef_file = "function_defs.py"
if os.path.isfile(nb_funcdef_file):
    from function_defs import listdir_contains, read_lightcurve, refine_flare_ranges, calculate_flare_energy
    from function_defs import is_left_censored, is_right_censored, is_peak_censored, peak_flux, peak_time
else:
    raise IOError("Could not find function definition file '" + nb_funcdef_file + "' that goes with this notebook.")

In [ ]:
# Restore the output directory.  Note: this assumes you've run the "generate_products" notebook already.  If not you
# will need to specify the location of the products made from the "generate_products" notebook.
%store -r data_directory
# If you have not run the "generate_products" notebook during this session, uncomment the line below and specify
# the location of the output products.
#data_directory = "./raw_files/"

In [ ]:
# Restore the distance parameter.  Note: this assumes you've run the "generate_products" notebook already.  If not you
# will need to specify the distane to use.
%store -r distance
# If you have not run the "generate_products" notebook during this session, uncomment the line below and specify
# the distance to the system in parsecs.
#distance = 1/(372.1631/1000) # parsecs

In [ ]:
# Locate the photon files.
photon_files = {'NUV':listdir_contains(data_directory,'nd-30s.csv'),
                'FUV':listdir_contains(data_directory,'fd-30s.csv')}

In [ ]:
# Make the table of flare properties in the paper.
flare_table = pd.DataFrame()
n_visits = 10
expts = []
for i in np.arange(n_visits):
    lc_nuv = read_lightcurve(photon_files['NUV'][i])
    expts += [len(lc_nuv)*30]
    lc_fuv = read_lightcurve(photon_files['FUV'][i])
    (flare_ranges, quiescence, quiescence_err) = refine_flare_ranges(lc_nuv, makeplot=False)
    (flare_ranges_fuv, quiescence_fuv, quiescence_err_fuv) = refine_flare_ranges(lc_fuv, makeplot=False)
    
    if i==8: # override the algorithmic detection for this visit, decomposing it into two flares
        flare_ranges = [[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17],
                        [17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]]
    for flare_range in flare_ranges:
        # Check that flux is simultaneously >3-sigma above quiescence in both bands
        nuv_3sig = np.array(flare_range)[np.where((np.array(lc_nuv['cps'].iloc[flare_range].values)-
                             3*np.array(lc_nuv['cps_err'].iloc[flare_range].values)>=quiescence))[0]].tolist()
        fuv_3sig = np.where((np.array(lc_fuv['cps'].values)-
                             3*np.array(lc_fuv['cps_err'].values)>=quiescence_fuv))[0]
        real = (any(set(nuv_3sig) & set(fuv_3sig)) or (len(nuv_3sig)>1)) # force detection conditions
        if not real:
            continue

        flare_data = {'visit_num':i,'flare_num':len(flare_table)+1,'duration':len(flare_range)*30}
        energy_nuv = calculate_flare_energy(lc_nuv, flare_range, distance, binsize=30, band='NUV',
                           quiescence=[quiescence,quiescence_err])
        energy_fuv = calculate_flare_energy(lc_fuv, flare_range, distance, binsize=30, band='FUV')
        nuv_sn = max(((np.array(lc_nuv['cps'].iloc[flare_range].values) -
                       3*np.array(lc_nuv['cps_err'].iloc[flare_range].values)) / quiescence))
        flare_data['energy_nuv'] = energy_nuv[0]
        flare_data['energy_err_nuv'] = energy_nuv[1]
        flare_data['energy_fuv'] = energy_fuv[0]
        flare_data['energy_err_fuv'] = energy_fuv[1]
        flare_data['nuv_sn'] = nuv_sn
        flare_data['left_censored'] = is_left_censored(flare_range)
        flare_data['right_censored'] = is_right_censored(lc_nuv,flare_range)
        flare_data['peak_flux_nuv'] = peak_flux(lc_nuv,flare_range)
        flare_data['peak_t0_nuv'] = peak_time(lc_nuv,flare_range)
        flare_data['peak_censored'] = is_peak_censored(lc_nuv,flare_range)
        flare_data['peak_flux_fuv'] = peak_flux(lc_fuv,flare_range)
        flare_data['peak_t0_fuv'] = peak_time(lc_fuv,flare_range)
        flare_data['quiescence_nuv'] = quiescence
        flare_data['quiescence_err_nuv'] = quiescence_err
        flare_table = flare_table.append(flare_data,ignore_index=True)
        # Make plots
        plt.figure(figsize=(16,4))
        plt.title(' visit #{i}, flare #{m}) -- '.format(real=real,i=i+1,m=len(flare_table)))
        plt.errorbar(lc_fuv['t0']-min(lc_nuv['t0']),
                         lc_fuv['flux_apcorrected'],
                         yerr=3*lc_fuv['flux_err'],fmt='rx-',alpha=0.5,label='FUV')
        plt.errorbar(lc_nuv['t0']-min(lc_nuv['t0']),
                         lc_nuv['flux_apcorrected'],
                         yerr=3*lc_nuv['flux_err'],fmt='kx-',label='NUV')
        plt.plot(lc_nuv['t0'].iloc[flare_range]-min(lc_nuv['t0']),
                     lc_nuv['flux_apcorrected'].iloc[flare_range],'bo',
                     label = '{e} +/- {e_err} log10(ergs)'.format(
                         e=np.round(np.log10(energy_nuv[0]),2),
                         e_err=np.round(np.log10(energy_nuv[1]),2)))
        plt.hlines(gt.counts2flux(quiescence,'NUV'),lc_nuv['t0'].min()-min(lc_nuv['t0']),
                                                        lc_nuv['t0'].max()-min(lc_nuv['t0']),
                       label='NUV quiescence',linestyles='dashed',color='r')
        plt.hlines(gt.counts2flux(quiescence_fuv,'FUV'),lc_nuv['t0'].min()-min(lc_nuv['t0']),
                                                            lc_nuv['t0'].max()-min(lc_nuv['t0']),
                       label='FUV quiescence',linestyles='dotted',color='b')
        t_buffer = (1050-flare_data['duration'])/2
        plt.xlim([lc_nuv['t0'].iloc[flare_range].min()-min(lc_nuv['t0'])-t_buffer,
                      lc_nuv['t1'].iloc[flare_range].max()-min(lc_nuv['t0'])+t_buffer])
        plt.ylim([max((lc_nuv['flux_apcorrected']-4*lc_nuv['flux_err']).iloc[flare_range].min(),
                          (lc_fuv['flux_apcorrected']-4*lc_fuv['flux_err']).iloc[flare_range].min()),
                      max((lc_nuv['flux_apcorrected']+4*lc_nuv['flux_err']).iloc[flare_range].max(),
                          (lc_fuv['flux_apcorrected']+4*lc_fuv['flux_err']).iloc[flare_range].max())])
        plt.legend()
        plt.xlabel('Seconds from start of visit')
        plt.ylabel('Flux units')
        plt.savefig('figures/flare_{m}.png'.format(m=len(flare_table)),dpi=400)

In [ ]:
# Reformat the energy measurements to include error bars and reasonable sigfigs
nuv_energy_string, fuv_energy_string = [], []
for e, e_err in zip(np.array(np.log10(flare_table['energy_nuv']), dtype='float16'),
                   np.array(np.log10(flare_table['energy_err_nuv']), dtype='float16')):
    nuv_energy_string += ['{:4.2f} pm {:4.2f}'.format(e, e_err, 2)]
for e, e_err in zip(np.array(np.log10(flare_table['energy_fuv']), dtype='float16'),
                   np.array(np.log10(flare_table['energy_err_fuv']), dtype='float16')):
    fuv_energy_string += ['{:4.2f} pm {:4.2f}'.format(e, e_err, 2)]

# Reformat the peak timestamps
t = pd.to_datetime(flare_table['peak_t0_nuv'] + gt.GPSSECS, unit='s')
t.iloc[np.where(np.array(flare_table['peak_censored'], dtype='bool'))] = 'peak not measured'

# Convert key columns in the flare table into a format suitable for printing in a LaTeX table
summary_table = pd.DataFrame({
    'Flare #':np.array(flare_table['flare_num'],dtype='int16'),
    'Visit #':np.array(flare_table['visit_num'],dtype='int16')+1,
    'NUV Peak Time (UTC)':t,
    'NUV S:N':flare_table['nuv_sn'],
    'Duration (min.)':np.array(flare_table['duration']/60.,dtype='float16'),
    'NUV Energy*':nuv_energy_string,
    'FUV Energy*':fuv_energy_string,
})
print(summary_table.to_latex(index=False))